<a href="https://colab.research.google.com/github/USC-Melady/KDD19-CoSTCo/blob/master/costco_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [0]:
import os
import json
import urllib2
import zipfile
from pprint import pprint

import numpy as np
import tensorflow as tf
import keras as k

# Load data

In [0]:
data_folder = 'data'

if not os.path.exists(data_folder):
  url = 'https://github.com/USC-Melady/KDD19-CoSTCo/releases/download/demo_data/data.zip'
  filedata = urllib2.urlopen(url)  

  with open('data.zip', 'wb') as f:  
      f.write(filedata.read())

  with zipfile.ZipFile('data.zip', 'r') as f:
    f.extractall('.')

In [0]:
shape = np.loadtxt(os.path.join(data_folder, 'tensor_shape.txt')).astype(int)
tr_idxs = np.loadtxt(os.path.join(data_folder, 'train_indices.txt')).astype(int)
tr_vals = np.loadtxt(os.path.join(data_folder, 'train_values.txt'))
te_idxs = np.loadtxt(os.path.join(data_folder, 'test_indices.txt')).astype(int)
te_vals = np.loadtxt(os.path.join(data_folder, 'test_values.txt'))

# Define util functions

In [0]:
def mape_keras(y_true, y_pred, threshold=0.1):
    v = k.backend.clip(k.backend.abs(y_true), threshold, None)
    diff = k.backend.abs((y_true - y_pred) / v)
    return 100.0 * k.backend.mean(diff, axis=-1)

def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return 100.0 * np.mean(diff, axis=-1).mean()

def transform(idxs):
    return [idxs[:, i] for i in range(idxs.shape[1])]

def set_session(device_count=None, seed=0):
    gpu_options = tf.GPUOptions(allow_growth=True)
    if device_count is not None:
        config = tf.ConfigProto(
            gpu_options=gpu_options, 
            device_count=device_count
        )
    else:
        config = tf.ConfigProto(gpu_options=gpu_options)
    sess = tf.Session(config=config)
    k.backend.set_session(sess)

    np.random.seed(seed)
    tf.set_random_seed(seed)
    return sess
  
def get_metrics(model, x, y, batch_size=1024):
    yp = model.predict(x, batch_size=batch_size, verbose=1).flatten()
    return {
        "rmse": float(rmse(y, yp)), 
        "mape": float(mape(y, yp)), 
        "mae": float(mae(y, yp))
    }

# Create a CoSTCo model 

In [0]:
def create_costco(shape, rank, nc):
    inputs = [k.Input(shape=(1,), dtype="int32") for i in range(len(shape))]
    embeds = [
        k.layers.Embedding(output_dim=rank, input_dim=shape[i])(inputs[i])
        for i in range(len(shape))
    ]
    x = k.layers.Concatenate(axis=1)(embeds)
    x = k.layers.Reshape(target_shape=(rank, len(shape), 1))(x)
    x = k.layers.Conv2D(
        nc, 
        kernel_size=(1, len(shape)), 
        activation="relu", 
        padding="valid"
    )(x)
    x = k.layers.Conv2D(
        nc, 
        kernel_size=(rank, 1), 
        activation="relu", 
        padding="valid"
    )(x)
    x = k.layers.Flatten()(x)
    x = k.layers.Dense(nc, activation="relu")(x)
    outputs = k.layers.Dense(1, activation="relu")(x)
    model = k.Model(inputs=inputs, outputs=outputs)

    return model

# Set hyper-parameters 

In [0]:
lr = 1e-4
rank = 20
nc = rank
epochs = 50
batch_size = 256

seed = 3
verbose = 1

# Train with early stopping

In [0]:
set_session(device_count={"GPU": 0}, seed=seed)
optim = k.optimizers.Adam(lr=lr)

model = create_costco(shape, rank, nc)
model.compile(optim, loss=["mse"], metrics=["mae", mape_keras])
hists = model.fit(
    x=transform(tr_idxs),
    y=tr_vals,
    verbose=verbose,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[k.callbacks.EarlyStopping(
        monitor="val_mean_absolute_error", 
        patience=10, 
        restore_best_weights=True)],
);

# Evaluation model prediction over the test set

In [0]:
tr_info = get_metrics(model, transform(tr_idxs), tr_vals)
te_info = get_metrics(model, transform(te_idxs), te_vals)

In [0]:
pprint({'train': tr_info, 'test': te_info})